<a href="https://colab.research.google.com/github/zishan7030/AdBuss/blob/main/delhi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Business Advisor for opening new Business***


In [2]:
!pip install beautifulsoup4

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import pandas as pd
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
from folium import * # plotting library
from bs4 import BeautifulSoup
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


**getting requred dataframe of List of neighbourhoods in Delhi. scraping web page**

In [3]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_of_Delhi").text
soup = BeautifulSoup(req, 'lxml')
div = soup.find('div', class_="mw-parser-output")
print("web Page Imported")
#Code to extract the relevent data from the request object using beautiful soup
data = pd.DataFrame(columns=['Borough','Neighborhood'])
i=-1
flag = False
no=0
prev_borough = None
for child in div.children:
    if child.name:
        span = child.find('span')
        if span!=-1 and span is not None:
            try:
                if span['class'][0] == 'mw-headline' and child.a.text!='edit':
                    prev_borough = child.a.text
                    i+=1
                    flag = True
                    continue
            except KeyError:
                continue
        if child.name=='ul' and flag==True:
            neighborhood = []
            for ch in child.children:
                
                try:
                    data.loc[no]=[prev_borough,ch.text]
                    no+=1
                except AttributeError:
                    pass
        flag = False
data[50:60]
dfn= pd.DataFrame(data)
dfn.shape

web Page Imported


(114, 2)

In [4]:
dfn.to_csv('/content/delhi_csv.csv', index=False)

Now,We Getting Location Data From Geolocator by providing List_of_neighbourhoods_in_Delhi data



In [5]:
lat_lng = pd.DataFrame(columns=['latitude','longitude'])
geolocator = Nominatim(user_agent="ny_explorer")
for i in range(114):
    address = data['Neighborhood'].loc[i]+',Delhi'
    try: 
        location = geolocator.geocode(address)
        lat_lng.loc[i]=[location.latitude,location.longitude]
    except AttributeError:
        continue
df1 = data
df2 = lat_lng
delhi_neighbourhood_data = pd.concat([df1, df2], axis=1)

In [6]:
delhi_neighbourhood_data.head()

,Borough,Neighborhood,latitude,longitude
0,North West Delhi,Adarsh Nagar,28.614193,77.071541
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Begum Pur,28.725503,77.058371
3,North West Delhi,Karala,28.735140,77.032511
4,North West Delhi,Model Town,28.702714,77.193991


In [7]:
delhi_neighbourhood_data.shape


(114, 4)

In [8]:
delhi_neighbourhood_data.to_csv('/content/delhi_lag_lat_csv.csv', index=False)


In [9]:

df1=pd.read_csv('/content/delhi_csv.csv')
df1.shape

(114, 2)

In [10]:
df1.head()


,Borough,Neighborhood
0,North West Delhi,Adarsh Nagar
1,North West Delhi,Ashok Vihar
2,North West Delhi,Begum Pur
3,North West Delhi,Karala
4,North West Delhi,Model Town


# **Analysis of neighbourhoods using map**
Creating Folium Map of List of neighbourhoods in Delhi

In [11]:

import folium
latitude = df2.loc[0, 'latitude']
longitude = df2.loc[0, 'longitude']
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map

for lat, lng, label in zip(df2['latitude'],df2['longitude'],df1['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_delhi)  
    
map_delhi 



Getting Data Using Foursquare API by providing ID and Secret.

In [12]:

CLIENT_ID = 'MGMBVNJKXHKVKCPQNXRD401BWMBZ4YQUF3JDHG5P3FEANLHR' # your Foursquare ID
CLIENT_SECRET = '10RLP4RHEHTAEXBIOOEP3NV1YYZQA3VMLVGDW3N1UDSQOFTA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MGMBVNJKXHKVKCPQNXRD401BWMBZ4YQUF3JDHG5P3FEANLHR
CLIENT_SECRET:10RLP4RHEHTAEXBIOOEP3NV1YYZQA3VMLVGDW3N1UDSQOFTA


In [13]:
latitude = df2.loc[0, 'latitude']
longitude = df2.loc[0, 'longitude']
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [14]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
from pandas.io.json import json_normalize

In [15]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Bikanerwala,Indian Restaurant,28.613391,77.076084
1,Uttam nagar,Metro Station,28.620201,77.068709
2,Gold's Gym A Block Janakpuri,Gym,28.622439,77.069348
3,Potholes at Dabri,Pool,28.605309,77.072504


In [16]:

nearby_venues.shape

(4, 4)

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
  

In [18]:
delhi_venues = getNearbyVenues(names =df1 ['Neighborhood'],
                               latitudes =df2 ['latitude'],
                               longitudes =df2 ['longitude'])

Adarsh Nagar
Ashok Vihar
Begum Pur
Karala
Model Town
Narela
Pitam Pura
Rohini Sub City
Shalimar Bagh
Civil Lines
Gulabi Bagh
Kamla Nagar
Kashmiri Gate
Kotwali
Mori Gate; Delhi
Sadar Bazaar
Sarai Rohilla
Shakti Nagar
Tis Hazari
Timarpur
Wazirabad
Dilshad Garden
Naveen Shahdara
New Usmanpur
Shahdara
Shastri Park
Yamuna Vihar
Chandni Chowk
Daryaganj
Jhandewalan
Karol Bagh
Shastri Nagar
Kishanganj
Paharganj
Rajender Nagar
Barakhamba Road
Chanakyapuri
Connaught Place
East Vinod Nagar
Jhilmil Colony
Laxmi Nagar
Mayur Vihar
Pandav Nagar
Preet Vihar
Anand Vihar
Vasundhara Enclave
Vishwas Nagar
Vivek Vihar
Alaknanda
Chattarpur
Chittaranjan Park
Defence Colony
East of Kailash
Govindpuri
Greater Kailash
Green Park
Gulmohar Park
Hauz Khas
Hauz Khas Village
Jangpura
Jor Bagh
Kalkaji
Khanpur
Lajpat Nagar
Lodi Colony
Malviya Nagar
Mehrauli
Neeti Bagh
Nehru Place
Netaji Nagar
New Friends Colony
Nizamuddin West
Okhla
Safdarjung Enclave
Sainik Farm
Saket
Sangam Vihar
Sarita Vihar
Sarojini Nagar
Sarvoday

In [19]:
print(delhi_venues.shape)
delhi_venues.head()

(2211, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adarsh Nagar,28.614193,77.071541,Bikanerwala,28.613391,77.076084,Indian Restaurant
1,Adarsh Nagar,28.614193,77.071541,Uttam nagar,28.620201,77.068709,Metro Station
2,Adarsh Nagar,28.614193,77.071541,Gold's Gym A Block Janakpuri,28.622439,77.069348,Gym
3,Adarsh Nagar,28.614193,77.071541,Potholes at Dabri,28.605309,77.072504,Pool
4,Ashok Vihar,28.699453,77.184826,Bellagio,28.696361,77.180021,Asian Restaurant


In [20]:
import pandas as pd

delhi_venues.to_csv('/content/delhi_venues.csv', index=False)

In [21]:
delhi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adarsh Nagar,4,4,4,4,4,4
Alaknanda,49,49,49,49,49,49
Anand Vihar,6,6,6,6,6,6
Ashok Nagar,13,13,13,13,13,13
Ashok Vihar,10,10,10,10,10,10
...,...,...,...,...,...,...
Vasundhara Enclave,8,8,8,8,8,8
Vishwas Nagar,26,26,26,26,26,26
Vivek Vihar,8,8,8,8,8,8


In [22]:
print('There are {} uniques categories.'.format(len(delhi_venues['Venue Category'].unique())))

There are 187 uniques categories.


# Creating Onehot encoding

In [23]:
delhi_onehot = pd.get_dummies(delhi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
delhi_onehot['Neighborhood'] = delhi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [delhi_onehot.columns[-1]] + list(delhi_onehot.columns[:-1])
delhi_onehot = delhi_onehot[fixed_columns]

delhi_onehot.head()

,Women's Store,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,...,Racetrack,Record Shop,Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
delhi_onehot.shape

(2211, 187)

In [25]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped

,Neighborhood,Women's Store,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,...,Racetrack,Record Shop,Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,Adarsh Nagar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040816,0.000,0.0,0.000000,0.0,0.040816,0.020408,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061224,0.0,0.020408,0.0,0.0,0.081633,0.0,0.0,0.0,...,0.0,0.0,0.000,0.102041,0.0,0.0,0.0,0.061224,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020408,0.0,0.020408,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Anand Vihar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Ashok Nagar,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.076923,0.0,0.0,0.153846,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Ashok Vihar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.100000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.000,0.000000,0.0,0.100000,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Vasundhara Enclave,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.125,0.0,0.125000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.125,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.125,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
104,Vishwas Nagar,0.0,0.000000,0.0,

# getting Freq of TOP 5 venues.

In [26]:
num_top_venues = 5

for hood in delhi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = delhi_grouped[delhi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adarsh Nagar----
               venue  freq
0      Metro Station  0.25
1               Pool  0.25
2                Gym  0.25
3  Indian Restaurant  0.25
4      Women's Store  0.00


----Alaknanda----
               venue  freq
0         Restaurant  0.10
1  Indian Restaurant  0.08
2        Coffee Shop  0.08
3             Market  0.06
4       Dessert Shop  0.06


----Anand Vihar----
           venue  freq
0            Gym  0.33
1    Pizza Place  0.17
2  Big Box Store  0.17
3  Shopping Mall  0.17
4          Hotel  0.17


----Ashok Nagar----
                            venue  freq
0               Indian Restaurant  0.15
1                     Coffee Shop  0.15
2  Multicuisine Indian Restaurant  0.08
3                     Pizza Place  0.08
4                             ATM  0.08


----Ashok Vihar----
                  venue  freq
0           Snack Place   0.1
1            Donut Shop   0.1
2    Light Rail Station   0.1
3           Pizza Place   0.1
4  Fast Food Restaurant   0.1


----Badar

# Getting top 10 venues of all neighbourhoods.

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = delhi_grouped['Neighborhood']

for ind in np.arange(delhi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(delhi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Indian Restaurant,Pool,Gym,Metro Station,Wings Joint,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
1,Alaknanda,Restaurant,Indian Restaurant,Coffee Shop,Dessert Shop,Pizza Place,Café,Sandwich Place,Market,Italian Restaurant,Fast Food Restaurant
2,Anand Vihar,Gym,Pizza Place,Hotel,Big Box Store,Shopping Mall,Electronics Store,Event Space,Garden,Furniture / Home Store,Diner
3,Ashok Nagar,Indian Restaurant,Coffee Shop,Pizza Place,ATM,Department Store,Food Truck,Hotel,Sandwich Place,Multicuisine Indian Restaurant,Fast Food Restaurant
4,Ashok Vihar,Pizza Place,Light Rail Station,Donut Shop,Fast Food Restaurant,Coffee Shop,Sandwich Place,South Indian Restaurant,Asian Restaurant,Snack Place,Indian Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
103,Vasundhara Enclave,BBQ Joint,Pool,Shop & Service,Athletics & Sports,Asian Restaurant,Metro Station,Track,Indian Restaurant,Fried Chicken Joint,Food & Drink Shop
104,Vishwas Nagar,Coffee Shop,Restaurant,Indian Restaurant,Market,BBQ Joint,Italian Restaurant,Food & Drink Shop,Gym,Thai Restaurant,Convenience Store
105,Vivek Vihar,Indian Restaurant,Café,Resort,Restaurant,Farm,Food Truck,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant
106,Wazirabad,Pizza Place,Shop & Service,Indian Restaurant,Light Rail Station,Metro Station,Food,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


# Lets Do clustering For neighbourhoods and venue 5 Cluster

In [29]:
from sklearn.cluster import KMeans

kclusters = 5

delhi_grouped_clustering = delhi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(delhi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 0, 0, 0, 2, 2, 2, 2, 2], dtype=int32)

In [30]:
neighborhoods_venues_sorted['Neighborhood']


0            Adarsh Nagar
1               Alaknanda
2             Anand Vihar
3             Ashok Nagar
4             Ashok Vihar
              ...        
103    Vasundhara Enclave
104         Vishwas Nagar
105           Vivek Vihar
106             Wazirabad
107          Yamuna Vihar
Name: Neighborhood, Length: 108, dtype: object

In [31]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,Indian Restaurant,Pool,Gym,Metro Station,Wings Joint,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
1,Alaknanda,Restaurant,Indian Restaurant,Coffee Shop,Dessert Shop,Pizza Place,Café,Sandwich Place,Market,Italian Restaurant,Fast Food Restaurant
2,Anand Vihar,Gym,Pizza Place,Hotel,Big Box Store,Shopping Mall,Electronics Store,Event Space,Garden,Furniture / Home Store,Diner
3,Ashok Nagar,Indian Restaurant,Coffee Shop,Pizza Place,ATM,Department Store,Food Truck,Hotel,Sandwich Place,Multicuisine Indian Restaurant,Fast Food Restaurant
4,Ashok Vihar,Pizza Place,Light Rail Station,Donut Shop,Fast Food Restaurant,Coffee Shop,Sandwich Place,South Indian Restaurant,Asian Restaurant,Snack Place,Indian Restaurant


In [32]:
delhi_merged = neighborhoods_venues_sorted

# add clustering labels
delhi_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

delhi_merged.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Adarsh Nagar,Indian Restaurant,Pool,Gym,Metro Station,Wings Joint,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,2
1,Alaknanda,Restaurant,Indian Restaurant,Coffee Shop,Dessert Shop,Pizza Place,Café,Sandwich Place,Market,Italian Restaurant,Fast Food Restaurant,2
2,Anand Vihar,Gym,Pizza Place,Hotel,Big Box Store,Shopping Mall,Electronics Store,Event Space,Garden,Furniture / Home Store,Diner,0
3,Ashok Nagar,Indian Restaurant,Coffee Shop,Pizza Place,ATM,Department Store,Food Truck,Hotel,Sandwich Place,Multicuisine Indian Restaurant,Fast Food Restaurant,0
4,Ashok Vihar,Pizza Place,Light Rail Station,Donut Shop,Fast Food Restaurant,Coffee Shop,Sandwich Place,South Indian Restaurant,Asian Restaurant,Snack Place,Indian Restaurant,0


# Appending Lag Lat dataframe to above dataset

In [53]:
lag_lat=pd.read_csv("//content/delhi_csv.csv")
dflag=df2['latitude']
dflat=df2['longitude']
lag_lat= pd.concat([dflag, dflat], axis=1)
#lag_lat.head()
delhi_sorted=pd.concat([neighborhoods_venues_sorted, lag_lat], axis=1)
delhi_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,latitude,longitude
0,Adarsh Nagar,Indian Restaurant,Pool,Gym,Metro Station,Wings Joint,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,2.0,28.614193,77.071541
1,Alaknanda,Restaurant,Indian Restaurant,Coffee Shop,Dessert Shop,Pizza Place,Café,Sandwich Place,Market,Italian Restaurant,Fast Food Restaurant,2.0,28.699453,77.184826
2,Anand Vihar,Gym,Pizza Place,Hotel,Big Box Store,Shopping Mall,Electronics Store,Event Space,Garden,Furniture / Home Store,Diner,0.0,28.725503,77.058371
3,Ashok Nagar,Indian Restaurant,Coffee Shop,Pizza Place,ATM,Department Store,Food Truck,Hotel,Sandwich Place,Multicuisine Indian Restaurant,Fast Food Restaurant,0.0,28.735140,77.032511
4,Ashok Vihar,Pizza Place,Light Rail Station,Donut Shop,Fast Food Restaurant,Coffee Shop,Sandwich Place,South Indian Restaurant,Asian Restaurant,Snack Place,Indian Restaurant,0.0,28.702714,77.193991


In [34]:
delhi_sorted.shape

(114, 14)


# **# DATA Analysis start**

# Creating Folium Map of Cluster we just define.

In [54]:
#map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
import matplotlib.cm as cm
from matplotlib import colors
latitude = delhi_sorted.loc[0, 'latitude']
longitude = delhi_sorted.loc[0, 'longitude']
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, poi,cluster in zip(df2['latitude'],df2['longitude'],df1['Neighborhood'],delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)  
    
map_clusters

# Show Data of common venue from each cluster

## *first* *cluster* *i*.*e* '*cluster* *0*'

In [36]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 0, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Gym,Shopping Mall,Electronics Store,Event Space,Garden,Furniture / Home Store,Diner,0
3,Indian Restaurant,Department Store,Food Truck,Hotel,Sandwich Place,Multicuisine Indian Restaurant,Fast Food Restaurant,0
4,Pizza Place,Coffee Shop,Sandwich Place,South Indian Restaurant,Asian Restaurant,Snack Place,Indian Restaurant,0
10,Indian Restaurant,Road,Movie Theater,Cricket Ground,Soccer Stadium,Metro Station,Asian Restaurant,0
14,BBQ Joint,Asian Restaurant,Athletics & Sports,Metro Station,Food & Drink Shop,Garden Center,Garden,0
16,Pizza Place,Food & Drink Shop,Burrito Place,Light Rail Station,Hotel,Fast Food Restaurant,Garden,0
17,Pizza Place,Donut Shop,Dumpling Restaurant,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,0
18,Pizza Place,Indian Restaurant,Flea Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,0
21,Hotel,Spa,Dumpling Restaurant,Department Store,Mughlai Restaurant,North Indian Restaurant,Bed & Breakfast,0
30,Indian Restaurant,Theater,Stadium,Chinese Restaurant,Metro Station,BBQ Joint,Flea Market,0


## *second* *cluster* *i*.*e* '*cluster* *1*'

In [37]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 1, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
32,ATM,Food & Drink Shop,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,1
82,ATM,Gastropub,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,1
93,ATM,Garden Center,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,1


## *Third* *cluster* *i*.*e* '*cluster* *2*'

In [38]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 2, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Indian Restaurant,Wings Joint,Food & Drink Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,2
1,Restaurant,Pizza Place,Café,Sandwich Place,Market,Italian Restaurant,Fast Food Restaurant,2
5,Indian Restaurant,Mediterranean Restaurant,Golf Course,Garden Center,French Restaurant,Outdoors & Recreation,Park,2
6,Train Station,Light Rail Station,Metro Station,Wings Joint,Food Court,Garden,Furniture / Home Store,2
7,Indian Restaurant,Chinese Restaurant,Coffee Shop,Lounge,BBQ Joint,Clothing Store,Pub,2
...,...,...,...,...,...,...,...,...
101,Metro Station,Fried Chicken Joint,Café,Sports Bar,Moving Target,BBQ Joint,Shopping Mall,2
102,Metro Station,Sports Bar,Café,Moving Target,Shopping Mall,Fast Food Restaurant,Pizza Place,2
103,BBQ Joint,Asian Restaurant,Metro Station,Track,Indian Restaurant,Fried Chicken Joint,Food & Drink Shop,2
104,Coffee Shop,BBQ Joint,Italian Restaurant,Food & Drink Shop,Gym,Thai Restaurant,Convenience Store,2


## *fourth* *cluster* *i*.*e* '*cluster* *3*'

In [39]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 3, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
60,IT Services,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,3


## *Fifth* *cluster* *i*.*e* '*cluster* *4*'

In [40]:
delhi_merged.loc[delhi_merged['Cluster Labels'] == 4, delhi_merged.columns[[1] + list(range(5, delhi_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
98,Market,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,4


# **Analysis result:**

# Lets Finding Best location or Neighborhood for new Business.

In [41]:
delhi_grouped = delhi_onehot.groupby('Neighborhood').mean().reset_index()
delhi_grouped

,Neighborhood,Women's Store,ATM,Accessories Store,Airport,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bistro,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,...,Racetrack,Record Shop,Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Tapas Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wings Joint
0,Adarsh Nagar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Alaknanda,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040816,0.000,0.0,0.000000,0.0,0.040816,0.020408,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061224,0.0,0.020408,0.0,0.0,0.081633,0.0,0.0,0.0,...,0.0,0.0,0.000,0.102041,0.0,0.0,0.0,0.061224,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020408,0.0,0.020408,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,Anand Vihar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Ashok Nagar,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.076923,0.0,0.0,0.153846,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.076923,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,Ashok Vihar,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.100000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.000,0.000000,0.0,0.100000,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Vasundhara Enclave,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125000,0.125,0.0,0.125000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.125,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.125,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
104,Vishwas Nagar,0.0,0.000000,0.0,

In [42]:
len(delhi_grouped[delhi_grouped["Coffee Shop"] > 0])
delhi_rest = delhi_grouped[["Neighborhood","Coffee Shop"]]
delhi_rest


,Neighborhood,Coffee Shop
0,Adarsh Nagar,0.000000
1,Alaknanda,0.081633
2,Anand Vihar,0.000000
3,Ashok Nagar,0.153846
4,Ashok Vihar,0.100000
...,...,...
103,Vasundhara Enclave,0.000000
104,Vishwas Nagar,0.192308
105,Vivek Vihar,0.000000
106,Wazirabad,0.000000


# **Clustering venue**

In [43]:
kclusters = 5

delhi_clustering = delhi_rest.drop(["Neighborhood"], 1)
delhi_clustering.head()

# run k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=12).fit(delhi_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 3, 2, 1, 3, 2, 2, 0, 2, 2], dtype=int32)


### **Getting Mean for each Neighborhoods of coffee shop.**

In [44]:
merged = delhi_rest.copy()

# add clustering labels
merged["Category"] = kmeans.labels_
#merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
merged.head()

,Neighborhood,Coffee Shop,Category
0,Adarsh Nagar,0.000000,2
1,Alaknanda,0.081633,3
2,Anand Vihar,0.000000,2
3,Ashok Nagar,0.153846,1
4,Ashok Vihar,0.100000,3


### **Create Folium Map to analys cluster**

In [47]:
lag_lat=pd.read_csv("//content/delhi_csv.csv")
dflag=df2['latitude']
dflat=df2['longitude']
lag_lat= pd.concat([dflag, dflat], axis=1)
#lag_lat.head()
delhi_w_Loc=pd.concat([merged, lag_lat], axis=1)
delhi_w_Loc.head()

,Neighborhood,Coffee Shop,Category,latitude,longitude
0,Adarsh Nagar,0.000000,2.0,28.614193,77.071541
1,Alaknanda,0.081633,3.0,28.699453,77.184826
2,Anand Vihar,0.000000,2.0,28.725503,77.058371
3,Ashok Nagar,0.153846,1.0,28.735140,77.032511
4,Ashok Vihar,0.100000,3.0,28.702714,77.193991


In [62]:
#map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
import matplotlib.cm as cm
from matplotlib import colors
latitude = delhi_w_Loc.loc[0, 'latitude']
longitude = delhi_w_Loc.loc[0, 'longitude']
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to map
for lat, lng, poi,cluster in zip(df2['latitude'],df2['longitude'],df1['Neighborhood'],merged['Category']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster -1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)  
    
map_clusters

In [56]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 0]
#Sort
delhi_w_Loc.sort_values(["Category"], inplace=True, ascending=False)
delhi_w_Loc

,Neighborhood,Coffee Shop,Category,latitude,longitude
78,Rohini Sub City,0.250000,4.0,28.574157,77.195370
104,Vishwas Nagar,0.192308,4.0,28.653281,77.141773
100,Tughlaqabad,0.200000,4.0,28.560691,77.160791
22,East Vinod Nagar,0.200000,4.0,28.677339,77.286016
92,Shastri Nagar,0.200000,4.0,28.667367,77.184438
...,...,...,...,...,...
109,NaN,NaN,NaN,28.645112,77.123933
110,NaN,NaN,NaN,28.634636,77.107112
111,NaN,NaN,NaN,28.639650,77.094039
112,NaN,NaN,NaN,28.644009,77.054470


Category 1: Neighborhoods with very low number of Coffee Shop

In [57]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 0.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
7,Barakhamba Road,0.050000,0.0,NaN,NaN
11,Chattarpur,0.037037,0.0,28.680344,77.202129
12,Chittaranjan Park,0.073171,0.0,28.669977,77.232059
55,Moti Nagar,0.062500,0.0,28.555537,77.202497
94,Siri Fort,0.071429,0.0,28.612304,76.982391
19,Dhaula Kuan,0.055556,0.0,28.701263,77.218680
21,Dwarka Sub City,0.050000,0.0,28.675826,77.321516
24,Fateh Nagar,0.055556,0.0,28.673333,77.289025
80,Safdarjung Enclave,0.063291,0.0,28.552146,77.224698
76,Rajender Nagar,0.041667,0.0,28.497702,77.239174



Category 2: Neighborhoods with low number of coffee shop

In [58]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 1.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
64,New Friends Colony,0.142857,1.0,28.590702,77.220921
27,Green Park,0.171429,1.0,28.655983,77.232194
86,Sarita Vihar,0.166667,1.0,28.610770,77.091106
88,Sarvodaya Enclave,0.150000,1.0,28.593833,77.134979
3,Ashok Nagar,0.153846,1.0,28.735140,77.032511


In [59]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 2.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
59,Narela,0.000000,2.0,28.583041,77.239317
75,Punjabi Bagh,0.000000,2.0,28.524411,77.213725
57,Najafgarh,0.000000,2.0,28.544256,77.206707
74,Preet Vihar,0.000000,2.0,28.503746,77.216073
73,Pitam Pura,0.000000,2.0,28.565642,77.193438
...,...,...,...,...,...
5,Badarpur,0.000000,2.0,28.842610,77.091835
6,Bali Nagar,0.000000,2.0,28.703268,77.132250
41,Khanpur,0.000000,2.0,28.613107,77.295722
40,Kashmiri Gate,0.027778,2.0,28.630553,77.277575


In [60]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 3.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
1,Alaknanda,0.081633,3.0,28.699453,77.184826
45,Lajpat Nagar,0.085106,3.0,28.601726,77.321122
29,Gulmohar Park,0.102564,3.0,28.644319,77.199917
56,Munirka,0.125000,3.0,28.557101,77.213005
26,Greater Kailash,0.090909,3.0,28.700372,77.272773
25,Govindpuri,0.089888,3.0,28.668434,77.250530
23,East of Kailash,0.096774,3.0,28.683855,77.256005
85,Sarai Rohilla,0.090909,3.0,28.493170,77.303024
35,Jhilmil Colony,0.083333,3.0,28.629318,77.225841
91,Shalimar Bagh,0.111111,3.0,28.544537,77.127923


In [61]:
delhi_w_Loc.loc[delhi_w_Loc['Category'] == 4.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
78,Rohini Sub City,0.250000,4.0,28.574157,77.195370
104,Vishwas Nagar,0.192308,4.0,28.653281,77.141773
100,Tughlaqabad,0.200000,4.0,28.560691,77.160791
22,East Vinod Nagar,0.200000,4.0,28.677339,77.286016
92,Shastri Nagar,0.200000,4.0,28.667367,77.184438
